In [1]:
from utils import *

In [2]:
config = Config().load(os.path.join("configs", "config.json"))

In [3]:
class MultiCrossEntropy(nn.Module):
    def __init__(self, padding):
        super().__init__()

        self.entropy = nn.CrossEntropyLoss(ignore_index=-100)
        self.padding = padding

    def forward(self, yPred, yTrue, split=False):
        loss = {}
        for key in yPred:
            target = yTrue[key]
            if self.padding[key] is not None:
                mask = target == self.padding[key]
                target = target.masked_fill(mask, -100)
                
            if torch.all(target == -100):
                continue

            loss[key] = self.entropy(yPred[key].permute(0, 2, 1), target)

        if not split:
            return sum(value for value in loss.values()) / len(loss)
        else:
            return loss

In [4]:
def iterate(dataset):
    while True:
        for batch in dataset:
            yield batch

def trainModel(config, modelClass, datasetClass, objective):
    model = modelClass(config.model).to(DEVICE)
    dataset = datasetClass(config.dataset, fixMissing=False)

    generator = torch.Generator(device=DEVICE)

    # TODO: Correct splits
    train, test = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=generator)
    train = DataLoader(train, batch_size=config.batchSize, shuffle=True, collate_fn=datasetClass.collate, generator=generator)
    test = DataLoader(test, batch_size=config.batchSize // 2, shuffle=True, collate_fn=datasetClass.collate, generator=generator)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learningRate)

    client = Client("127.0.0.1", 12945)

    testIter = iterate(test)
    
    progress = 0

    for epoch in range(config.epochs):
        for batch in train:
            model.train()
            optimizer.zero_grad()

            sequences = batch["sequences"]
            inputs = {key: value[:, :-1] for key, value in sequences.items()}
            targets = {key: value[:, 1:] for key, value in sequences.items()}

            outputs = model(inputs)
            loss = objective(outputs, targets)

            loss.backward()
            optimizer.step()

            with torch.no_grad():
                model.eval()

                batch1 = next(testIter)
                sequences1 = batch1["sequences"]
                inputs1 = {key: value[:, :-1] for key, value in sequences1.items()}
                targets1 = {key: value[:, 1:] for key, value in sequences1.items()}

                outputs1 = model(inputs1)
                loss1 = objective(outputs1, targets1)

            client.send("Train Loss", loss.item())
            client.send("Test Loss", loss1.item())

            splitTrain = objective(outputs, targets, split=True)
            splitTest = objective(outputs1, targets1, split=True)
            for key in splitTrain:
                client.send(f"Train {key} Loss", splitTrain[key].item())
            for key in splitTest:
                client.send(f"Test {key} Loss", splitTest[key].item())
                
            print(f"\r{epoch + 1} | {progress + 1}/{len(train)} | {loss.item():.4f} | {loss1.item():.4f}", end="")
            progress += 1

In [5]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
trainModel(config, MIDIK, LakhData, MultiCrossEntropy(config.padding))

TypeError: MIDIK.__init__.<locals>.<lambda>() missing 1 required positional argument: 'x'